In [1]:
import numpy
from scipy.spatial.transform import Rotation
import plotly.graph_objects as go

In [2]:
def from_two_vectors(a, b):
    a = a / numpy.linalg.norm(a)
    b = b / numpy.linalg.norm(b)
    v = numpy.cross(a, b)
    s = numpy.linalg.norm(v)
    c = a.dot(b)
    K = numpy.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    return numpy.eye(3) + K + (1 - c) / s**2 * K @ K


def rotate_to_z(n):
    return from_two_vectors(n, numpy.array([0, 0, 1]))


def rotate_around_z(theta):
    return numpy.array([[numpy.cos(theta), -numpy.sin(theta), 0],
                        [numpy.sin(theta),
                         numpy.cos(theta), 0], [0, 0, 1]])


def decompose_to_z_screwing(R0, R1):
    # Decompose the inbetween rotation as a rotation around the z-axis:
    #     R = Pᵀ R_z P
    # Where R = R₁R₀ᵀ, P is a rotation from n̂ to ẑ, and R_z is a rotation
    # of ω around the z-axis.
    r = Rotation.from_matrix(R1 @ R0.T).as_rotvec()
    omega = numpy.linalg.norm(r)
    P = rotate_to_z(r / omega)
    return P, omega

In [3]:
r0 = numpy.array([0, 1, 0])
R0 = Rotation.from_rotvec(r0).as_matrix()
dr = numpy.array([2, 3, 1])
p0 = numpy.array([0, 0, 0])
dp = numpy.array([0, 3, 0])
v = numpy.array([1, 0, 0])

In [4]:
def generate_line_search_trajectory(alpha):
    R1 = Rotation.from_rotvec(r0 + alpha * dr).as_matrix()
    P, omega = decompose_to_z_screwing(R0, R1)

    # Compute trajectory of a point
    v0_to_1 = [
        P.T @ rotate_around_z(omega * t) @ P @ R0 @ v + (alpha * dp * t + p0)
        for t in numpy.linspace(0, 1, 100)
    ]
    return numpy.vstack(v0_to_1)

In [5]:
def generate_line_search_trajectory2(alpha):
    # Compute trajectory of a point
    v0_to_1 = [
        Rotation.from_rotvec(r0 + t * alpha * dr).as_matrix() @ v +
        (alpha * dp * t + p0) for t in numpy.linspace(0, 1, 1000)
    ]
    return numpy.vstack(v0_to_1)

In [13]:
data = []
for i in range(0, 10):
    points = generate_line_search_trajectory(1 / 2**i)
    data.append(
        go.Scatter3d(x=points[:, 0],
                     y=points[:, 1],
                     z=points[:, 2],
                     mode='lines',
                     name=f"alpha=1/2^{i}"))
fig = go.Figure(data=data)
fig.show()

In [11]:
data = []
for i in range(0, 10):
    points = generate_line_search_trajectory2(1 / 2**i)
    data.append(
        go.Scatter3d(x=points[:, 0],
                     y=points[:, 1],
                     z=points[:, 2],
                     mode='lines',
                     name=f"alpha=1/2^{i}"))
fig = go.Figure(data=data)
fig.show()

In [8]:
data = []
points = generate_line_search_trajectory(1)
data.append(
    go.Scatter3d(x=points[:, 0],
                 y=points[:, 1],
                 z=points[:, 2],
                 mode='lines',
                 name="Screwing [Redon et al. 2002]"))
points = generate_line_search_trajectory2(1)
data.append(
    go.Scatter3d(x=points[:, 0],
                 y=points[:, 1],
                 z=points[:, 2],
                 mode='lines',
                 name="Linear interpolation of rotation vectors"))
fig = go.Figure(data=data)
fig.show()

In [9]:
points = generate_line_search_trajectory2(1)
y = numpy.linalg.norm(points[0:-1, :] - points[1:, :], axis=1)
fig = go.Figure(data=[
    go.Scatter(y=y, mode='lines'),
    go.Scatter(x=[0, y.size - 1], y=y[[0, -1]], mode='lines')
])
fig.show()